In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from requests_html import HTMLSession
from time import sleep
import os

TCCB Turkish

In [ ]:
isExist = os.path.exists("SpeechesTR\\Turkish")
if not isExist:
   # Create a new directory because it does not exist
   os.makedirs("SpeechesTR\\Turkish")
   os.makedirs("SpeechesTR\\English")

df = pd.DataFrame(columns = ["date", "title", "file", "link"])
count = 1

for page_num in range(1, 33):
    if (page_num == 1):
        url = "https://www.tccb.gov.tr/receptayyiperdogan/konusmalar/"
    else:
        url = "https://www.tccb.gov.tr/receptayyiperdogan/konusmalar/?&page=" + str(page_num)
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'lxml')
    list_press_address = soup.find("div", {"id": "list-press-address"})
    dl_data = list_press_address.find_all("dl")
    for dlitem in dl_data:
        speech_date = dlitem.find_all("dt")[0].text
        speech_title = dlitem.find_all("dd")[0].text
        speech_url = "https://www.tccb.gov.tr" + dlitem.find_all("a")[0].get("href")
        speech_page = requests.get(speech_url)
        speech_soup = BeautifulSoup(speech_page.text, 'lxml')
        speech_by_p = speech_soup.find("div", {"id": "divContentArea"})
        if (speech_by_p is None):
            print(speech_title)
            continue
        speech_text = ""
        for p in speech_by_p.find_all("p"):
            speech_text += p.text + " "
        
        file_name = "PresSpeech" + str(count) + ".txt"
        with open("SpeechesTR\\Turkish\\" + file_name, 'w', encoding='mbcs') as f:
            f.write(speech_text)
        to_append = [speech_date, speech_title, file_name, speech_url]
        count += 1

        a_series = pd.Series(to_append, index = df.columns)
        df = df.append(a_series, ignore_index=True)

    print(page_num, "COMPLETED")
    sleep(2)
    df.to_csv("PresSpeechesTR.csv", index=False)
print(df.shape[0])

In [ ]:
df = pd.read_csv("PresSpeechesTR.csv")
print(df.shape[0])
sum = 0
count = 0
max = 0
min = 100000
for index, row in df.iterrows():
    text_file = open(df[index, "file"], "r", encoding="mbcs")
    speech = (text_file.read())
    count += 1
    speech_length = len(speech)
    sum += speech_length
    if (speech_length > max):
        max = speech_length
    if (speech_length < min):
        min = speech_length
print("Average character count", sum/count)
print("Max character count", min)
print("Min character count", max)

In [ ]:
# Imports the Google Cloud client library
from google.cloud import storage
from nltk import tokenize

def translate_text(target, text):
    """Translates text into the target language.

    Target must be an ISO 639-1 language code.
    See https://g.co/cloud/translate/v2/translate-reference#supported_languages
    """
    import six
    from google.cloud import translate_v2 as translate

    translate_client = translate.Client.from_service_account_json(
        "universal-sun-342204-0d31a6d61733.json")

    if isinstance(text, six.binary_type):
        text = text.decode("utf-8")

    # Text can also be a sequence of strings, in which case this method
    # will return a sequence of results for each text.
    result = translate_client.translate(text, target_language=target, source_language="tr")
    # print(u"Text: {}".format(result["input"]))
    # print(u"Translation: {}".format(result["translatedText"]))
    # print(u"Detected source language: {}".format(result["detectedSourceLanguage"]))
    # print(result["translatedText"])
    return result["translatedText"]

df = pd.read_csv("PresSpeechesTR.csv")
df.insert(2, "translated_title", "") 
df = df.reset_index()  # make sure indexes pair with number of rows
for index, row in df.iterrows():
    text_file = open(df[index, "file"], "r", encoding="mbcs")
    speech = (text_file.read())
    list_sentences = tokenize.sent_tokenize(speech)
    translated = ""
    to_translate = ""
    for sentence in list_sentences:
        if (len(to_translate) + len(sentence) < 5000):
            to_translate += sentence + " "
        else:
            translated += translate_text("en", to_translate) + " "
            to_translate = ""
    translated += translate_text("en", to_translate) + " "
    with open("SpeechesTR\\English\\" + df[index, "file"], 'w', encoding='utf-8') as f:
        f.write(translated)
    df[index, "translated_title"] = translate_text("en", row["title"])
df.to_csv("PresSpeechesTR.csv", index=False)

TCCB English

In [ ]:
# "22.08.2022"
# range(1, 17)

isExist = os.path.exists("SpeechesEN")
if not isExist:
   os.makedirs("SpeechesEN")

df = pd.DataFrame(columns = ["date", "title", "file", "link"])
count = 1
found = False
for page_num in range(1, 17):
    if (page_num == 1):
        url = "https://www.tccb.gov.tr/en/receptayyiperdogan/speeches/"
    else:
        url = "https://www.tccb.gov.tr/en/receptayyiperdogan/speeches/?&page=" + str(page_num)
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'lxml')
    list_press_address = soup.find("div", {"id": "list-press-address"})
    dl_data = list_press_address.find_all("dl")
    for dlitem in dl_data:
        speech_date = dlitem.find_all("dt")[0].text
        if speech_date == "22.08.2022":
            found = True
        if not found:
            continue
        speech_title = dlitem.find_all("dd")[0].text
        speech_url = "https://www.tccb.gov.tr" + dlitem.find_all("a")[0].get("href")
        speech_page = requests.get(speech_url)
        speech_soup = BeautifulSoup(speech_page.text, 'lxml')
        speech_by_p = speech_soup.find("div", {"id": "divContentArea"})
        if (speech_by_p is None):
            print(speech_title)
            continue
        speech_text = ""
        for p in speech_by_p.find_all("p"):
            speech_text += p.text + " "
        
        file_name = "PresSpeechEN" + str(count) + ".txt"
        with open("SpeechesEN\\" + file_name, 'w', encoding='utf-8') as f:
            f.write(speech_text)
        to_append = [speech_date, speech_title, file_name, speech_url]
        count += 1

        a_series = pd.Series(to_append, index = df.columns)
        df = df.append(a_series, ignore_index=True)

    print(page_num, "COMPLETED")
    sleep(2)
    df.to_csv("PresSpeechesEN.csv", index=False)
print(df.shape[0])

Video Gallery in Turkish

In [20]:
import wget
isExist = os.path.exists("Videos")
if not isExist:
   os.makedirs("Videos")

# count = 1
df = pd.DataFrame(columns = ["title", "file", "url"])

for i in range(1, 245):
    url = "https://www.tccb.gov.tr/canliyayin/?Keyword=&page=" + str(i) + "#Video"
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'lxml')
    search_results_div = soup.find("div", {"id": "SearchResult"})
    video_divs = search_results_div.find_all("div", {"class": "video-gallery-area"})

    for video_div in video_divs:
        video_spot = video_div.find("div", {"class": "video-spot"})
        video_url = "https://www.tccb.gov.tr"
        video_url += video_spot.find("img")['src'].replace("resim/vposter", "video").replace(".jpg", ".mp4")
        # r = requests.get(video_url, stream = True) 
        # file_name = "Video" + str(count) + ".mp4"
        # count += 1
        # download started 
        # with open("Videos\\" + file_name, 'wb') as f: 
        #     for chunk in r.iter_content(chunk_size = 1024*1024): 
        #         if chunk: 
        #             f.write(chunk) 

        file_name = wget.download(video_url, out="\\Videos\\")
        
        video_title = video_div.find("div", {"class": "video-title"}).find("h2").text.strip()
        to_append = [video_title, file_name, video_url]
        a_series = pd.Series(to_append, index = df.columns)
        df = df.append(a_series, ignore_index=True)
    print("Page", i, "completed.")
    df.to_csv("PresVideos.csv", index=False)

In [9]:
df = pd.DataFrame(columns = ["title", "url", "date"])

for i in range(1, 245):
    url = "https://www.tccb.gov.tr/canliyayin/?Keyword=&page=" + str(i) + "#Video"
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'lxml')
    search_results_div = soup.find("div", {"id": "SearchResult"})
    video_divs = search_results_div.find_all("div", {"class": "video-gallery-area"})

    for video_div in video_divs:
        video_spot = video_div.find("div", {"class": "video-spot"})
        video_url = "https://www.tccb.gov.tr"
        video_url += video_spot.find("img")['src'].replace("resim/vposter", "video").replace(".jpg", ".mp4")
        
        video_title = video_div.find("div", {"class": "video-title"}).find("h2").text.strip()
        date = ""
        date_list = video_url.split("/")[-1].split("-")
        if len(date_list) >= 3:
            year, month, day = date_list[0], date_list[1], date_list[2]
            date = day + "." + month + "." + year
        to_append = [video_title, video_url, date]
        a_series = pd.Series(to_append, index = df.columns)
        df = df.append(a_series, ignore_index=True)
    if i % 10 == 0:
        print("Page", i, "completed.")
        sleep(2)

df.to_csv("PresVideoLinks.csv", index=False)

Page 10 completed.
Page 20 completed.
Page 30 completed.
Page 40 completed.
Page 50 completed.
Page 60 completed.
Page 70 completed.
Page 80 completed.
Page 90 completed.
Page 100 completed.
Page 110 completed.
Page 120 completed.
Page 130 completed.
Page 140 completed.
Page 150 completed.
Page 160 completed.
Page 170 completed.
Page 180 completed.
Page 190 completed.
Page 200 completed.
Page 210 completed.
Page 220 completed.
Page 230 completed.
Page 240 completed.


In [12]:
import wget

already_downloaded = os.listdir("Videos")
df = pd.read_csv("PresVideoLinks.csv")
for url in df["url"]:
    if url.split("/")[-1] not in already_downloaded:
        file_name = wget.download(url, out= os.getcwd() + "\\Videos\\")
        print(file_name)
# df["url"][0].split("/")[-1]